<a href="https://colab.research.google.com/github/freakezoide/codigos-de-coolab/blob/main/celulares_con_graficas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
import pandas as pd

# Esto abrirá una ventana para que selecciones tu archivo Excel
uploaded = files.upload()

# Una vez subido, el archivo estará disponible en el entorno de Colab
# y podrás ver su nombre
for fn in uploaded.keys():
  print('Archivo subido: "{name}" con un tamaño de {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Reemplaza 'tu_archivo.xlsx' con el nombre real de tu archivo subido
# que viste en el mensaje anterior
file_name = list(uploaded.keys())[0] # Esto toma el nombre del primer archivo subido
df = pd.read_excel(file_name)

print("\n¡Archivo Excel cargado exitosamente!")

Saving Mobiles Dataset (2025).xlsx to Mobiles Dataset (2025).xlsx
Archivo subido: "Mobiles Dataset (2025).xlsx" con un tamaño de 92662 bytes

¡Archivo Excel cargado exitosamente!


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier # O KNeighborsRegressor si es regresión
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score

# Assuming 'target_column_name' is the name of your target column in the Excel file
# **REPLACE 'target_column_name' WITH THE ACTUAL NAME OF YOUR TARGET COLUMN**
# For example, if your target column is named 'Precio', change the line below:
target_column_name = 'Marca' # <--- CHANGE THIS LINE

# Separate features (X) and target (y) from the DataFrame
# Identify columns to drop: the target column and any other non-numeric columns
columns_to_drop = [target_column_name]

# Add other non-numeric columns to the list if necessary
# For example, if 'Modelo' is a non-numeric column you want to exclude:
if 'Modelo' in df.columns:
    columns_to_drop.append('Modelo')
# You might need to add other column names here if they are non-numeric and not useful features

# Drop the specified columns to get the initial feature set X
X = df.drop(columns=columns_to_drop)
y = df[target_column_name]

# --- NEW CHANGE: Select only numeric columns from X ---
# This ensures that only columns compatible with StandardScaler are kept
X = X.select_dtypes(include=['number'])

# Check the data types of the columns in X to confirm they are all numeric
print("Data types of feature columns (X) after selecting numeric types:")
print(X.dtypes)

# Check if X is empty after dropping non-numeric columns
if X.empty:
    print("Error: No numeric features remaining after removing non-numeric columns.")
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"\nTamaño del conjunto de entrenamiento (X): {X_train.shape}")
    print(f"Tamaño del conjunto de prueba (X): {X_test.shape}")
    print(f"Tamaño del conjunto de entrenamiento (y): {y_train.shape}")
    print(f"Tamaño del conjunto de prueba (y): {y_test.shape}\n")

    # --- PASO 2: Escalado de características ---
    # Crea una instancia del escalador
    scaler = StandardScaler()

    # Ajusta el escalador a los datos de entrenamiento y transforma ambos conjuntos
    # This should now work as X_train should only contain numeric data
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print("Características escaladas.\n")

    # --- PASO 3: Entrenamiento del modelo KNN ---

    # Decide si es clasificación o regresión y elige el modelo adecuado:
    # Para CLASIFICACIÓN:
    # k_neighbors: El número de vecinos a considerar (un hiperparámetro importante)
    n_neighbors = 5 # Puedes probar con diferentes valores (ej. 3, 5, 7, etc.)
    knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)

    # Para REGRESIÓN (si 'y' es numérica):
    # knn_model = KNeighborsRegressor(n_neighbors=n_neighbors)

    # Entrena el modelo con los datos escalados
    knn_model.fit(X_train_scaled, y_train)

    print(f"Modelo KNN (con k={n_neighbors}) entrenado.\n")

    # --- PASO 4: Predicción ---
    y_pred = knn_model.predict(X_test_scaled)

    print("Predicciones realizadas en el conjunto de prueba.\n")

    # --- PASO 5: Evaluación del modelo ---

    # Para CLASIFICACIÓN:
    if isinstance(knn_model, KNeighborsClassifier):
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Precisión (Accuracy): {accuracy:.4f}")
        print("\nReporte de clasificación:")
        print(classification_report(y_test, y_pred))

    # Para REGRESIÓN:
    elif isinstance(knn_model, KNeighborsRegressor):
        mse = mean_squared_error(y_test, y_pred)
        rmse = mse**0.5
        r2 = r2_score(y_test, y_pred)
        print(f"Error Cuadrático Medio (MSE): {mse:.4f}")
        print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.4f}")
        print(f"Coeficiente de Determinación (R^2): {r2:.4f}")

Data types of feature columns (X) after selecting numeric types:
Peso del Movil               float64
RAM                          float64
Capacidad de Bateria         float64
Precio Lanzamiento (USA)     float64
Precio Lanzamiento(Dubai)    float64
dtype: object

Tamaño del conjunto de entrenamiento (X): (744, 5)
Tamaño del conjunto de prueba (X): (186, 5)
Tamaño del conjunto de entrenamiento (y): (744,)
Tamaño del conjunto de prueba (y): (186,)

Características escaladas.

Modelo KNN (con k=5) entrenado.

Predicciones realizadas en el conjunto de prueba.

Precisión (Accuracy): 0.4247

Reporte de clasificación:
              precision    recall  f1-score   support

       Apple       0.68      0.74      0.71        23
      Google       0.00      0.00      0.00         4
       Honor       0.31      0.50      0.38        20
      Huawei       0.20      0.14      0.17         7
     Infinix       0.11      0.29      0.16         7
      Lenovo       0.00      0.00      0.00         3
 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
import pandas as pd



# --- 1. Tabla de Frecuencia de Marcas (Conteo de cada Marca) ---
print("### 1. Tabla de Frecuencia de Marcas (Conteo de cada Marca)\n")
marca_counts = df['Marca'].value_counts()
print(marca_counts.to_markdown(numalign="left", stralign="left"))
print("\nEsta tabla muestra cuántas veces aparece cada marca en tu dataset, ordenada de mayor a menor frecuencia.\n")

# --- 2. Tabla de Porcentaje de Marcas (Distribución Relativa) ---
print("### 2. Tabla de Porcentaje de Marcas (Distribución Relativa)\n")
marca_percentages = (df['Marca'].value_counts(normalize=True) * 100).round(2) # Redondea a 2 decimales
print(marca_percentages.to_markdown(numalign="left", stralign="left"))
print("\nEsta tabla muestra el porcentaje de cada marca en relación con el total de móviles, redondeado a dos decimales.\n")

# --- 3. Tabla Descriptiva por Marca para Columnas Numéricas (Media, Mediana, Desviación Estándar) ---
print("### 3. Tabla Descriptiva por Marca para Columnas Numéricas (Media, Mediana, Desviación Estándar)\n")
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
# Excluir columnas que no son características numéricas útiles para agregación si las hay
# Por ejemplo, si tuvieras un 'ID' o 'Año' que no quieres promediar.
# numeric_cols = [col for col in numeric_cols if col not in ['ID_del_movil', 'Año_lanzamiento']] # Ajusta según tus datos

if 'Marca' in df.columns and numeric_cols:
    marca_descriptive = df.groupby('Marca')[numeric_cols].agg(['mean', 'median', 'std', 'min', 'max'])
    # Para mejorar la visualización en Markdown, puedes aplanar los multi-índices de columnas si lo deseas
    # marca_descriptive.columns = ['_'.join(col).strip() for col in marca_descriptive.columns.values]
    print(marca_descriptive.to_markdown(numalign="left", stralign="left"))
    print("\nEsta tabla muestra estadísticas descriptivas (media, mediana, desviación estándar, mínimo, máximo) de las características numéricas para cada marca.\n")
else:
    print("\nNo hay columnas numéricas adecuadas o la columna 'Marca' no está presente para agrupar.\n")

# --- 4. Tabla de las Top N Marcas por Precio Promedio (Ejemplo: Top 5) ---
print("### 4. Tabla de las Top N Marcas por Precio Promedio (Ejemplo: Top 5 por 'Precio Lanzamiento (USA)')\n")
if 'Precio Lanzamiento (USA)' in df.columns:
    top_brands_by_price = df.groupby('Marca')['Precio Lanzamiento (USA)'].mean().nlargest(5).round(2)
    print(top_brands_by_price.to_markdown(numalign="left", stralign="left"))
    print("\nEsta tabla muestra las 5 marcas con el precio de lanzamiento promedio más alto en USA.\n")
else:
    print("\nLa columna 'Precio Lanzamiento (USA)' no está presente.\n")

# --- 5. Tabla Cruzada (Crosstab) de Marcas y otra Variable (Ej. Rangos de RAM) ---
# Primero, vamos a crear una columna categórica para RAM para hacer una crosstab más significativa
# Creamos rangos de RAM:
bins = [0, 4, 8, 12, 16, 24, 32, 64] # Define tus propios rangos si es necesario
labels = ['<4GB', '4-8GB', '8-12GB', '12-16GB', '16-24GB', '24-32GB', '32GB+'] # Etiquetas para los rangos
if 'RAM' in df.columns:
    df['RAM_Categoria'] = pd.cut(df['RAM'], bins=bins, labels=labels, right=False, include_lowest=True)

    print("### 5. Tabla Cruzada (Crosstab) de Marcas y Rangos de RAM\n")
    # fill_value=0 para que donde no haya intersección aparezca 0 en lugar de NaN
    crosstab_marca_ram = pd.crosstab(df['Marca'], df['RAM_Categoria'], margins=True, margins_name='Total').fillna(0).astype(int)
    print(crosstab_marca_ram.to_markdown(numalign="left", stralign="left"))
    print("\nEsta tabla cruzada muestra la distribución de marcas por diferentes categorías de RAM, incluyendo totales por fila y columna.\n")
    # Opcional: Eliminar la columna temporal si no la necesitas más
    df = df.drop(columns=['RAM_Categoria'])
else:
    print("\nLa columna 'RAM' no está presente para crear categorías de RAM y la tabla cruzada.\n")

# --- 6. Tabla de Datos Crudos Filtrados (Ej. Marcas "Apple" o "Samsung") ---
print("### 6. Tabla de Datos Crudos Filtrados (Ej. solo móviles de 'Apple' o 'Samsung')\n")
# Puedes añadir más marcas a la lista o cambiar los criterios de filtrado
filtered_df = df[df['Marca'].isin(['Apple', 'Samsung'])]
# Muestra las primeras 10 filas del DataFrame filtrado para no saturar la salida
print(filtered_df.head(10).to_markdown(index=False, numalign="left", stralign="left"))
print("\nEsta tabla muestra las primeras 10 filas de los datos crudos, pero solo para móviles de las marcas 'Apple' o 'Samsung'.\n")

# --- 7. Tabla con Columnas Seleccionadas y Ordenadas ---
print("### 7. Tabla con Columnas Seleccionadas y Ordenadas (Ej. Marca, Precio, RAM, ordenado por Precio)\n")
selected_cols = ['Marca', 'Precio Lanzamiento (USA)', 'RAM', 'Capacidad de Bateria'] # Elige las columnas que te interesen
if all(col in df.columns for col in selected_cols):
    sorted_table = df[selected_cols].sort_values(by='Precio Lanzamiento (USA)', ascending=False).head(15) # Top 15 por precio
    print(sorted_table.to_markdown(index=False, numalign="left", stralign="left"))
    print("\nEsta tabla muestra un subconjunto de columnas (Marca, Precio, RAM, Batería), ordenado por el precio de lanzamiento en USA (descendente), mostrando los 15 primeros registros.\n")
else:
    print("\nUna o más de las columnas seleccionadas no están presentes en el DataFrame.\n")


### 1. Tabla de Frecuencia de Marcas (Conteo de cada Marca)

| Marca    | count   |
|:---------|:--------|
| Oppo     | 129     |
| Apple    | 97      |
| Honor    | 91      |
| Samsung  | 88      |
| Vivo     | 86      |
| Realme   | 69      |
| Motorola | 62      |
| Infinix  | 56      |
| OnePlus  | 53      |
| Huawei   | 42      |
| Tecno    | 39      |
| POCO     | 30      |
| Xiaomi   | 27      |
| Google   | 21      |
| Lenovo   | 15      |
| Nokia    | 11      |
| Sony     | 9       |
| iQOO     | 3       |
| Poco     | 2       |

Esta tabla muestra cuántas veces aparece cada marca en tu dataset, ordenada de mayor a menor frecuencia.

### 2. Tabla de Porcentaje de Marcas (Distribución Relativa)

| Marca    | proportion   |
|:---------|:-------------|
| Oppo     | 13.87        |
| Apple    | 10.43        |
| Honor    | 9.78         |
| Samsung  | 9.46         |
| Vivo     | 9.25         |
| Realme   | 7.42         |
| Motorola | 6.67         |
| Infinix  | 6.02         |
| OnePl